In [2]:
#This is a code cell in a Jupyter notebook
print("Hello, world!")

Hello, world!


In [3]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images().images
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1/255)(images)

/Users/mehannioui/Documents/HOML/practice/homl/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
images.shape

TensorShape([2, 70, 120, 3])

In [5]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding='same')
fmaps = conv_layer(images)

In [6]:
fmaps.shape

TensorShape([2, 70, 120, 32])

In [7]:
kernels, bianas = conv_layer.get_weights()
kernels.shape

(7, 7, 3, 32)

In [8]:
bianas.shape

(32,)

In [9]:
maxpool2d = tf.keras.layers.MaxPool2D(pool_size=2)

In [10]:
class DepthPool(tf.keras.layers.Layer):
  def __init__(self, pool_size=2, **kwargs):
    super().__init__(**kwargs)
    self.pool_size = pool_size

    def call(self, inputs):
      shape = tf.shape(inputs) # shape[-1] is the number of channels
      groups = shape[-1] // self.pool_size # number of channel groups
      new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis=0)
      return tf.reduce_max(lambda X: tf.reshape(inputs, new_shape), axis=-1)

In [11]:
global_avg_pool = tf.keras.layers.Lambda(
  lambda X: tf.reduce_mean(X, axis=[1,2])
)

In [12]:
global_avg_pool(images)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.64338624, 0.5971759 , 0.5824972 ],
       [0.76306933, 0.2601113 , 0.10849128]], dtype=float32)>

In [13]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3,
                      padding='same', activation='relu',
                      kernel_initializer='he_normal')

model = tf.keras.Sequential([
  DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
  tf.keras.layers.MaxPool2D(),  
  DefaultConv2D(filters=128),
  DefaultConv2D(filters=128),
  tf.keras.layers.MaxPool2D(),
  DefaultConv2D(filters=256),
  DefaultConv2D(filters=256),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=128, activation='relu',
                        kernel_initializer='he_normal'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(units=64, activation='relu',
                        kernel_initializer='he_normal'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(units=10, activation='softmax')
])


In [14]:
# Implementing a ResNet-34 CNN Using Keras

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3,
                        strides = 1, padding="same",
                        kernel_initializer = "he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self, filters, strides=1, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.main_layers = [
      DefaultConv2D(filters, strides=strides),
      tf.keras.layers.BatchNormalization(),
      self.activation,
      DefaultConv2D(filters),
      tf.keras.layers.BatchNormalization(),
    ]
    self.skip_layer = []
    if strides > 1:
      self.skip_layer = [
        DefaultConv2D(filters, kernel_size=1, strides=strides),
        tf.keras.layers.BatchNormalization()
      ]
  
  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layer:
      skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)

In [ ]:
# Building ResNet-34 using Sequential model
model = tf.keras.Sequential([
  DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
])
prev_filters = 64
for filter in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filter == prev_filters else 2
  model.add(ResidualUnit(filter, strides=strides))
  prev_filters = filter

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))